In [5]:
import cv2
import numpy as np
import os
from pathlib import Path
from tqdm import tqdm

## Mixed species

## Generate data for training 

In [9]:
def generate_mixed_darkfield_image(
    img_size=(512, 512),
    num_spheres=30,
    num_rods=20,
    sphere_radius_range=(6, 10),
    rod_size_range=((15, 5), (30, 8)),  # (min_l,w), (max_l,w)
    output_dir='darkfield_mixed/images',
    annotation_dir='darkfield_mixed/labels',
    image_id=0
):
    img = np.zeros(img_size, dtype=np.uint8)
    boxes = []

    # --- Spherical Cells (Class 0) ---
    for _ in range(num_spheres):
        r = np.random.randint(*sphere_radius_range)
        x = np.random.randint(r, img_size[1] - r)
        y = np.random.randint(r, img_size[0] - r)

        cv2.circle(img, (x, y), r, 255, 2)
        if np.random.rand() > 0.5:
            cv2.circle(img, (x, y), r - 1, 180, -1)

        cx, cy, w, h = x / img_size[1], y / img_size[0], (2 * r) / img_size[1], (2 * r) / img_size[0]
        boxes.append(f"0 {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

    # --- Rod-Shaped Bacteria (Class 1) ---
    for _ in range(num_rods):
        l = np.random.randint(rod_size_range[0][0], rod_size_range[1][0])
        w = np.random.randint(rod_size_range[0][1], rod_size_range[1][1])
        x = np.random.randint(l, img_size[1] - l)
        y = np.random.randint(w, img_size[0] - w)
        angle = np.random.randint(0, 180)

        # Draw ellipse to simulate rod
        cv2.ellipse(img, (x, y), (l // 2, w // 2), angle, 0, 360, 255, 2)
        if np.random.rand() > 0.5:
            cv2.ellipse(img, (x, y), (l // 2 - 1, w // 2 - 1), angle, 0, 360, 180, -1)

        # Convert bounding box
        w_rot, h_rot = get_rotated_bbox(l, w, angle)
        cx, cy = x / img_size[1], y / img_size[0]
        bbox_w = w_rot / img_size[1]
        bbox_h = h_rot / img_size[0]
        boxes.append(f"1 {cx:.6f} {cy:.6f} {bbox_w:.6f} {bbox_h:.6f}")

    # Light blur for realism
    img = cv2.GaussianBlur(img, (3, 3), 0)

    # Save image and label
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    Path(annotation_dir).mkdir(parents=True, exist_ok=True)
    img_path = f"{output_dir}/mix_{image_id:04d}.jpg"
    label_path = f"{annotation_dir}/mix_{image_id:04d}.txt"

    cv2.imwrite(img_path, img)
    with open(label_path, 'w') as f:
        f.write('\n'.join(boxes))

In [10]:
def get_rotated_bbox(length, width, angle):
    """Estimate a rotated bounding box size (axis-aligned)"""
    theta = np.radians(angle)
    w_rot = abs(length * np.cos(theta)) + abs(width * np.sin(theta))
    h_rot = abs(length * np.sin(theta)) + abs(width * np.cos(theta))
    return w_rot, h_rot

In [11]:
# Generate dataset
for i in range(500):  # or more
    generate_mixed_darkfield_image(image_id=i)

## Generate dataset for testing

In [20]:
def generate_darkfield_image(image_size=512, num_cells=10, out_path="darkfield_mixed/test/images", filename="test1.jpg"):
    img = np.zeros((image_size, image_size), dtype=np.uint8)

    for _ in range(num_cells):
        shape_type = np.random.choice(['circle', 'rod'])

        if shape_type == 'circle':
            radius = np.random.randint(8, 15)
            x = np.random.randint(radius, image_size - radius)
            y = np.random.randint(radius, image_size - radius)
            cv2.circle(img, (x, y), radius, (255,), -1)

        elif shape_type == 'rod':
            length = np.random.randint(20, 40)
            width = np.random.randint(6, 10)
            angle = np.random.uniform(0, 180)
            center = (np.random.randint(50, image_size - 50), np.random.randint(50, image_size - 50))
            box = cv2.boxPoints(((center[0], center[1]), (length, width), angle))
            box = np.int0(box)
            cv2.drawContours(img, [box], 0, (255,), -1)

    # Add Gaussian blur to simulate optics
    img = cv2.GaussianBlur(img, (5, 5), 1)

    # Save the image
    os.makedirs(out_path, exist_ok=True)
    cv2.imwrite(os.path.join(out_path, filename), img)

In [21]:
# Generate 10 new test images
for i in tqdm(range(10)):
    generate_darkfield_image(filename=f"test{i}.jpg")

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\desmo\AppData\Local\Temp\ipykernel_14064\1323145116.py:19: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 474.09it/s]
